In [ ]:
fig.show()

In [2]:
import pandas as pd, numpy as np, geopandas as gpd, os, sys
import plotly.express as px
from shapely import wkt
import plotly.graph_objects as go
from shapely.ops import nearest_points
from shapely.geometry import LineString
from arcgis.features import FeatureLayerCollection, FeatureLayer
from arcgis.gis import GIS
from arcgis.geometry import Geometry
from arcgis.geometry.filters import contains, overlaps, crosses, intersects

mca_pl_url = 'https://montgomeryplans.org/arcgis4/rest/services/Cached_overlays/property_poly_with_data/MapServer'
mca_add_url = 'https://montgomeryplans.org/arcgis4/rest/services/Overlays/Property_Address_Anno/MapServer'
mca_prop_url = 'https://montgomeryplans.org/arcgis4/rest/services/Overlays/MCAtlas_Property_Points/FeatureServer'

pl_gdf = gpd.read_file(r'zip://(POWER)_SHP_Deliverable\Property_Lines.zip')
df = pd.DataFrame([list(l.coords) for l in pl_gdf.geometry])
df[['x1', 'y1']] = pd.DataFrame(df[0].to_list()) * 0.3048006096012192
df[['x2', 'y2']] = pd.DataFrame(df[1].to_list()) * 0.3048006096012192

df['ls'] = [wkt.loads(f"LINESTRING ({r.x1} {r.y1}, {r.x2} {r.y2})") for i,r in df.iterrows()]

pl_gdf = gpd.GeoDataFrame(data = pl_gdf, geometry = df.ls, crs = 32618)
df = pd.DataFrame([list(l.coords) for l in pl_gdf.to_crs(4326).geometry])
df[['lon1', 'lat1']] = pd.DataFrame(df[0].to_list())
df[['lon2', 'lat2']] = pd.DataFrame(df[1].to_list())

pl_gdf = pl_gdf.merge(df.drop(columns = [0,1]), left_index = True, right_index = True)
all_proplines = pl_gdf.to_crs(2248).unary_union

problem_threshold_distance = 8

structure_df = pd.read_excel('69060_Segments 4-5_Pole Data Sheet_REV-B_2020-07-31.xlsx')
structure_df[['xm', 'ym']] = structure_df[['PROPOSED SURVEY COORD. X', 'PROPOSED SURVEY COORD. Y']] * 0.3048006096012192

structure_df = gpd.GeoDataFrame(data = structure_df, geometry = gpd.points_from_xy(structure_df.xm, structure_df.ym), crs = 32618).to_crs(4326)
structure_df['lat'] = structure_df.geometry.y
structure_df['lon'] = structure_df.geometry.x

structure_df['min_distance'] = [nearest_points(p, all_proplines)[0].distance(nearest_points(p, all_proplines)[1]) for p in structure_df.to_crs(2248).geometry]
problem_pole_df = structure_df[structure_df.min_distance < problem_threshold_distance]
# problem_pole_df.to_excel('69060_S4-5_problem_poles.xlsx')
circuit_df = pd.DataFrame(LineString(structure_df.geometry.unary_union).coords.xy).T

df = pd.DataFrame(pl_gdf.to_crs(3857).geometry.unary_union.convex_hull.buffer(0).boundary.coords.xy).T
filter_gdf = gpd.GeoDataFrame(data = [], geometry = gpd.points_from_xy(df[0], df[1]), crs = 3857).to_crs(4326)
filter_gdf['lon'] = filter_gdf.geometry.x
filter_gdf['lat'] = filter_gdf.geometry.y

geo_filter = contains(Geometry.from_shapely(
                pl_gdf.to_crs(2248).geometry.unary_union.convex_hull.buffer(100),
                spatial_reference = {'wkid' : 2248}
                ))

fc = FeatureLayerCollection(mca_prop_url)
sdf = fc.layers[0].query(geometry_filter = geo_filter).sdf

prop_gdf = gpd.GeoDataFrame(data = sdf.drop(columns = 'SHAPE'), geometry = sdf.SHAPE, crs = 2248)
prop_gdf = gpd.GeoDataFrame(data = prop_gdf.copy().drop(columns = 'geometry'), geometry = gpd.points_from_xy(prop_gdf.geometry.x, prop_gdf.geometry.y, crs = 2248)).to_crs(4326)
addr_cols = ['OWNERS_NAME', 'PREMISE_ADDR_HOUSENO', 'PREMISE_ADDR_DIRECTION', 'PREMISE_ADDR_STREET', 'PREMISE_ADDR_STREET_TYPE', 'PREMISE_ADDR_CITY', 'PREMISE_ADDR_ZIP']
repl_list = ['0', 0, np.nan, None]

prop_gdf['FULL_ADDRESS'] = [' '.join(r.to_list()).strip() for i,r in prop_gdf[addr_cols].replace(repl_list, '').astype('str').iterrows()]
prop_gdf['lon'] = prop_gdf.geometry.x
prop_gdf['lat'] = prop_gdf.geometry.y

traces = []

traces.append(
    go.Scattermapbox(
        name = 'New Poles',
        lon = structure_df.lon,
        lat = structure_df.lat,
        marker = dict(size = 5, color = 'cyan'),
        hoverinfo = 'text',
        text = structure_df['STRUCTURE NUMBER'],
        textposition = 'top right',
        mode = 'markers+text',
        textfont = dict(size = 18, color = 'cyan')
    )
)

traces.append(
    go.Scattermapbox(
        name = 'Problem Poles',
        lon = problem_pole_df.lon,
        lat = problem_pole_df.lat,
        marker = dict(size = 36, color = 'red'),
        hoverinfo = 'text',
        hovertext = problem_pole_df['min_distance'],
        text = problem_pole_df['STRUCTURE NUMBER'],
        textposition = 'bottom left',
        mode = 'markers+text',
        textfont = dict(size = 24, color = 'red')
    )
)

lons = np.empty(3 * len(pl_gdf))
lons[::3] = pl_gdf.lon1
lons[1::3] = pl_gdf.lon2
lons[2::3] = None
lats = np.empty(3 * len(pl_gdf))
lats[::3] = pl_gdf.lat1
lats[1::3] = pl_gdf.lat2
lats[2::3] = None

traces.append(
    go.Scattermapbox(
        name = 'Surveyed Property Lines',
        lon = lons,
        lat = lats,
        mode = 'lines',
        line = dict(width = 3, color = 'orange'),
        opacity = 1,
        hoverinfo = 'none',
        text = pl_gdf.index
    )
)

# fig.add_trace(
#     go.Scattermapbox(
#         name = 'MC Atlas Prop Line',
#         lon = propline_filter_df[0],
#         lat = propline_filter_df[1],
#         mode = 'lines',
#         line = dict(width = 1, color = 'orange'),
#         opacity = 1,
#         hoverinfo = 'none',
#         text = ''
#     )
# )

traces.append(
    go.Scattermapbox(
        name = 'Property Owner/Address',
        lon = prop_gdf['lon'],
        lat = prop_gdf['lat'],
        mode = 'text',
        # line = dict(width = 15, color = 'green'),
        opacity = 1,
        hoverinfo = 'text',
        text = prop_gdf['OWNERS_NAME'],
        hovertext = prop_gdf['FULL_ADDRESS'],
        textfont = dict(size = 20, color = 'magenta')

    )
)

traces.append(
    go.Scattermapbox(
        name = 'Alignment',
        lon = circuit_df[0],
        lat = circuit_df[1],
        mode = 'lines',
        line = dict(width = 5, color = 'lime'),
        opacity = 1,
        hoverinfo = 'none'
    )
)

fig = go.Figure(
    data = traces,
    layout = dict(
        mapbox_style="satellite-streets", mapbox_accesstoken = 'pk.eyJ1IjoiZXhjLWNsZSIsImEiOiJja2JmMzBzbW8wcnVzMnlvaWZydjc5d3l0In0.Lxvnw7hCMl-9f3smZtCFUg',
        height = 900,
        width = 1600,
        margin={"r":0,"t":0,"l":0,"b":0},
        mapbox = dict(center = go.layout.mapbox.Center(lon = -77.22768, lat = 39.16471), zoom = 18)
        )
    )

KeyError: "['SHAPE'] not found in axis"

#### ARCGIS IMPLEMENTATION FOR MC ATLAS LAYER IMPORT

structure_layer = gis.content.import_data(structure_df.drop(columns = 'geometry'))
testmap = gis.map('Montgomery County, MD', 13)

testmap.add_layer({"type":"FeatureLayer",
                "url":'https://montgomeryplans.org/arcgis4/rest/services/Cached_overlays/property_poly_with_data/MapServer',
               })
testmap.add_layer({"type":"FeatureLayer",
                "url":'https://mcatlas.org/arcgis3/rest/services/overlays/Address_Labels/MapServer/0',
               })
testmap.add_layer({"type":"FeatureLayer",
                "url":'https://mcatlas.org/arcgis3/rest/services/overlays/Plat_Index/MapServer',
               })
testmap.add_layer(structure_layer)
testmap.layout.height = '800px'

##### testmap

In [ ]:
def create_line_map(gdf, crs) :
    df = pd.DataFrame()
    new_gdf = gpd.GeoDataFrame()
    gdf_latlon = gpd.GeoDataFrame()
    try : geometries = gdf.geometry
    except :
        try : geometries = gdf.SHAPE
        except : print('no geometry/shape column passed!')

    new_gdf = gpd.GeoDataFrame(data = [], geometry = geometries, crs = crs).to_crs(4326)
    for b in new_gdf.geometry.boundary :
        df = pd.DataFrame([l.coords.xy for l in b.geoms][0]).T
        gdf_latlon = gdf_latlon.append(df)
        gdf_latlon = latlon_df.append([None])
    return gdf_latlon


In [ ]:
# gdf1 = prop_gdf.copy()
# gdf2 = problem_pole_df.copy()
# gdf1['wkt'] = [p.wkt for p in gdf1.geometry]

# match_df = pd.DataFrame([nearest_points(p, gdf1.geometry.unary_union) for i,p in gdf2.geometry.iteritems()], index = gdf2.index)
# match_df['wkt'] = [p.wkt for p in match_df[1]]

# problem_properties_gdf = gpd.GeoDataFrame(
#         gdf2.drop(columns = 'geometry')
#             .merge(match_df, left_index = True, right_index = True)
#             .merge(gdf1, left_on = 'wkt', right_on = 'wkt')
#             , crs = 4326)

# problem_properties_gdf['PREMISE_ADDR_ZIP'] = problem_properties_gdf.PREMISE_ADDR_ZIP.astype('Int64')
# problem_properties_gdf['lon'] = problem_properties_gdf.geometry.x
# problem_properties_gdf['lat'] = problem_properties_gdf.geometry.y


# problem_properties_gdf['FULL_ADDRESS'] = [' '.join(r.to_list()).strip() for i,r in problem_properties_gdf[addr_cols].replace(repl_list, '').astype('str').iterrows()]

In [ ]:
# pd.DataFrame(
#         [
#             [l.coords[0][0]
#             , l.coords[0][1]
#             , l.coords[1][0]
#             , l.coords[1][1]
#         ] for l in pl_gdf.geometry]
#     )
# # pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_aa_flight_paths.csv')